# Mosaicking Apertif HI Data

### This notebook is for developing the needed mosaicking algorithm to properly account for the covariance of the noise between beams.  

### This notebook is identical to the continuum version, except for the importing of the cubes instead of continuum maps.

#### June 6, 2019 D.J. Pisano


For starters, we will use the Apercal environment to work with data.  These tests are using the Lockman Hole data from observation 190428055.

In [ ]:
# Import needed packages
import os;
import shutil
import glob
import apercal.libs.lib as lib
import apercal
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from astropy.time import Time
import subprocess

myusername = os.environ['USER']
if not ('PYTHONPATH' in os.environ and
        '/home/' + myusername + '/apercal' in os.environ['PYTHONPATH']):
      print("WARNING: your apercal directory should be in the $PYTHONPATH")

%config IPCompleter.greedy=True
%matplotlib notebook
lib.setup_logger('info', logfile='/home/{}/testing/logs/mosaic_HI.log'.format(myusername))

cfgfile='/home/{}/testing/cfg/mosaic.cfg'.format(myusername)

metadir='/home/{}/metadata/'.format(myusername)   # This directory is where the beam map and correlation matrix reside.


%matplotlib inline



In [ ]:
# Get needed values from configuration file for where data is and where it is going.
prepare=apercal.prepare(cfgfile)

basedir=prepare.basedir
projectid=prepare.prepare_date+prepare.prepare_obsnum_target

linedir=basedir+projectid+'/line'

imagedir=linedir+'/raw/'

mosaicdir=linedir+'/'+prepare.mossubdir+'/'
beamdir=linedir+'/beams/'
#noisedir=basedir+projectid+'noise/'

# Create directories as needed.
if os.path.isdir(basedir+projectid)==False:
    os.mkdir(basedir+projectid)
if os.path.isdir(linedir)==False:
    os.mkdir(linedir)
if os.path.isdir(imagedir)==False:
    os.mkdir(imagedir)
if os.path.isdir(mosaicdir)==False:
    os.mkdir(mosaicdir)
if os.path.isdir(beamdir)==False:
    os.mkdir(beamdir)



## We assume that we have a beam model/models and a correlation matrix already generated and saved.  

## We can now start importing data.

In [ ]:
# Copy all images from all individual beams to "mosaic" directory

for beam in range(40):
    os.mkdir(imagedir+'{}'.format(str(beam).zfill(2)))
    if beam<10:
        os.system('cp /data/apertif/{0}/{1}/line/cubes/HI_image_cube_contsub.fits '.format(projectid,str(beam).zfill(2))+imagedir+'{}/.'.format(str(beam).zfill(2)))
    elif beam<20:
        os.system('cp /data2/apertif/{0}/{1}/line/cubes/HI_image_cube_contsub.fits '.format(projectid,str(beam).zfill(2))+imagedir+'{}/.'.format(str(beam).zfill(2)))
    elif beam<30:
        os.system('cp /data3/apertif/{0}/{1}/line/cubes/HI_image_cube_contsub.fits '.format(projectid,str(beam).zfill(2))+imagedir+'{}/.'.format(str(beam).zfill(2)))
    elif beam<40:
        os.system('cp /data4/apertif/{0}/{1}/line/cubes/HI_image_cube_contsub.fits '.format(projectid,str(beam).zfill(2))+imagedir+'{}/.'.format(str(beam).zfill(2)))


In [ ]:
# Import images/beams into Miriad


def import_image(beam_num):
    # This function will import a FITS image into Miriad placing it in the mosaicdir
    fits = lib.miriad('fits')
    fits.op = 'xyin'
    fits.in_ = imagedir+'{}/HI_image_cube_contsub.fits'.format(str(beam_num).zfill(2))
    fits.out = imagedir+'image_{}.map'.format(str(beam_num).zfill(2))
    fits.inp()
    fits.go()
    
def import_beam(beam_num):
    # This function will import the FITS image of a beam into Miriad format, placing it in the mosaicdir
    fits = lib.miriad('fits')
    fits.op = 'xyin'
    fits.in_ = metadir+'beam_{}.fits'.format(str(beam_num).zfill(2))
    fits.out = beamdir+'beam_{}.map'.format(str(beam_num).zfill(2))
    fits.inp()
    fits.go()
    
def duplicate_import_beam(beam_num):
    fits = lib.miriad('fits')
    fits.op = 'xyin'
    #fits.in_ = metadir+'cos6_beam.fits'
    fits.in_ = metadir+'gauss_beam.fits'
    fits.out = beamdir+'beam_{}.map'.format(str(beam_num).zfill(2))
    fits.inp()
    fits.go()
    
def valid_beam(bm_num):
    # This function will determine if there is a valid image for a given beam
    if os.path.exists(imagedir+'{}/HI_image_cube_contsub.fits'.format(str(bm_num).zfill(2))):
        return True
    else:
        return False



In [ ]:
# Import images from good beams 
beams=[]
for bm_num in range(40):
    if valid_beam(bm_num):
        beams.append(bm_num)
        
    
print(beams)

for beam in beams:
    import_image(beam)
    #import_beam(beam)
    duplicate_import_beam(beam)

In [ ]:
# Function to measure noise in all beams

def beam_noise(bm_num):
    sigest = lib.miriad('sigest')
    sigest.in_ = imagedir+'image_{}.map'.format(str(bm_num).zfill(2))
    return sigest.go()



In [ ]:
# Read in noise correlation matrix 
noise_cor=np.loadtxt(metadir+'correlation.txt',dtype='f')

# Initialize covariance matrix
noise_cov=noise_cor

# Measure noise in the image for each beam
sigma_beam=np.zeros(40,float)
for bm in beams:
    sigma_beam[bm]=float(beam_noise(bm)[4].lstrip('Estimated rms is '))
    
for a in beams:
    for b in beams:
        noise_cov[a,b]=noise_cor[a,b]*sigma_beam[a]*sigma_beam[b]  # The noise covariance matrix is 


    
# Only the inverse of this matrix is ever used:
inv_cov=np.linalg.inv(noise_cov)

print(noise_cov)
print(inv_cov)

In [ ]:
print(sigma_beam)

In [ ]:
# Transfer reference coordinates from images to associated beams
for beam in beams:
    gethd = lib.miriad('gethd')
    gethd.in_ = imagedir+'image_{}.map/crval1'.format(str(beam).zfill(2))
    ra1=gethd.go()
    puthd = lib.miriad('puthd')
    puthd.in_ = beamdir+'beam_{}.map/crval1'.format(str(beam).zfill(2))
    puthd.value = float(ra1[0])
    puthd.go()
    gethd.in_ = imagedir+'image_{}.map/crval2'.format(str(beam).zfill(2))
    dec1=gethd.go()
    puthd.in_ = beamdir+'beam_{}.map/crval2'.format(str(beam).zfill(2))
    puthd.value = float(dec1[0])
    puthd.go()


In [ ]:
# Get info in order to convolve images to same synthesized beam size

# Extract beam parameters from headers
bmaj=[]
bmin=[]
bpa=[]
for beam in beams:
    gethd = lib.miriad('gethd')
    gethd.in_ = imagedir+'image_{}.map/bmaj'.format(str(beam).zfill(2))
    bmaj.append(gethd.go())
    gethd.in_ = imagedir+'image_{}.map/bmin'.format(str(beam).zfill(2))
    bmin.append(gethd.go())
    gethd.in_ = imagedir+'image_{}.map/bpa'.format(str(beam).zfill(2))
    bpa.append(gethd.go())
    
# Calculate maximum bmaj and bmin and median bpa for final convolved beam shape
bmajor = [float(x[0]) for x in bmaj]
bmajor = 3600.*np.degrees(bmajor)

bminor = [float(x[0]) for x in bmin]
bminor = 3600.*np.degrees(bminor)

bangle = [float(x[0]) for x in bpa]
bangle = np.degrees(bangle)

c_beam = [1.05*np.nanmax(bmajor),1.05*np.nanmax(bminor),np.nanmedian(bangle)]
print('The final, convolved, synthesized beam has bmaj, bmin, bpa of: ',c_beam)

In [ ]:
# Convolve all images to same synthesized beam (c_beam)

for beam in beams:
    convol=lib.miriad('convol')
    convol.map = imagedir+'image_{}.map'.format(str(beam).zfill(2))
    convol.out = mosaicdir+'image_{}_convol.map'.format(str(beam).zfill(2))
    convol.fwhm = '{0},{1}'.format(str(c_beam[0]),str(c_beam[1]))
    convol.pa = c_beam[2]
    convol.options = 'final'
    convol.inp()
    convol.go()

In [ ]:
# Create template mosaic image using default parameters 

# Extract central RA and Dec for Apertif pointing
gethd = lib.miriad('gethd')
gethd.in_ = imagedir+'image_00.map/crval1'
gethd.format = 'hms'
ra_ref=gethd.go()
gethd.in_ = imagedir+'image_00.map/crval2'
gethd.format = 'dms'
dec_ref=gethd.go()

def create_mosaic():
    # This will create a template for the mosaic using "imgen" in Miriad
    imsize=5121     # Number of pixels for mosaic map
    cell=4.         # Cell size in arcsec
    # create template
    imgen = lib.miriad('imgen')
    imgen.out = mosaicdir+'mosaic_temp.map'
    imgen.imsize = imsize
    imgen.cell = cell
    imgen.object = 'level'
    imgen.spar = '0.'
    imgen.radec = '{0},{1}'.format(str(ra_ref[0]),str(dec_ref[0]))
    imgen.inp()
    imgen.go()
    # Now change projection to NCP
    regrid = lib.miriad('regrid')
    regrid.in_ = mosaicdir+'mosaic_temp.map'
    regrid.out = mosaicdir+'mosaic_template.map'
    regrid.project='NCP'
    regrid.go()
    shutil.rmtree(mosaicdir+'mosaic_temp.map')
    


In [ ]:
print('Reference RA, Dec for mosaic are: ',ra_ref,dec_ref)
create_mosaic()


In [ ]:
# Put images on mosaic template grid
for beam in beams:
    regrid = lib.miriad('regrid')
    regrid.in_ = mosaicdir+'image_{}_convol.map'.format(str(beam).zfill(2))
    regrid.out = mosaicdir+'image_{}_mos.map'.format(str(beam).zfill(2))
    regrid.tin = mosaicdir+'mosaic_template.map'
    regrid.axes = '1,2'
    regrid.inp()
    regrid.go()



In [ ]:
# Make beam maps match grid of images

for beam in beams:
    regrid = lib.miriad('regrid')
    regrid.in_ = beamdir+'beam_{}.map'.format(str(beam).zfill(2))
    regrid.out = beamdir+'beam_{}_mos.map'.format(str(beam).zfill(2))
    regrid.tin = mosaicdir+'mosaic_template.map'.format(str(beam).zfill(2))
    regrid.axes = '1,2'  
    regrid.inp()
    regrid.go()


In [ ]:
# Now, we need to do the matrix math

# First calculate transpose of beam matrix multiplied by the inverse covariance matrix
# Will use *maths* in Miriad

# Using "beams" list to account for missing beams/images
# Only doing math where inv_cov value is non-zero

maths = lib.miriad('maths')
for bm in beams:
    for b in beams:
        maths.out = mosaicdir+'tmp_{}.map'.format(str(b))
        if inv_cov[b,bm]!=0.:
                operate+="<"+beamdir+"beam_{0}_mos.map>*{1}+".format(str(b).zfill(2),inv_cov[b,bm])
        maths.exp = operate
        maths.options='unmask'
        maths.inp()
        maths.go()
    i=1
    while i<len(beams):
        if i==1:
            operate = "'<"+mosaicdir+"tmp_{}.map>+<".format(str(beams[i-1]))+mosaicdir+"tmp_{}.map>'".format(str(beams[i]))
        else:
            operate="'<"+mosaicdir+"tmp_{}.map>".format(str(beams[i]))+"+<"+mosaicdir+"sum_{}.map>'".format(str(beams[i-1]))
        maths.out = mosaicdir+'sum_{}.map'.format(str(beams[i]))
        maths.exp = operate
        maths.options='unmask'
        maths.inp()
        maths.go()
        i+=1
        
    os.rename(mosaicdir+'sum_{}.map'.format(str(beams[i-1])),mosaicdir+'btci_{}.map'.format(str(bm)))

    for fl in glob.glob(mosaicdir+'tmp_*.map'):
        shutil.rmtree(fl)
    for fl in glob.glob(mosaicdir+'sum_*.map'):
        shutil.rmtree(fl)


In [ ]:
# Calculate variance map (using beams and noise covariance matrix over entire map)
# This is the denominator for I(mosaic)

maths = lib.miriad('maths')
i=0
for bm in beams:
    operate="'<"+mosaicdir+"btci_{}.map>*<".format(str(bm).zfill(2))+beamdir+"beam_{}_mos.map>'".format(str(bm).zfill(2))
    if bm!=beams[0]:
        operate=operate[:-1]+"+<"+mosaicdir+"out_{}_mos.map>'".format(str(i).zfill(2))
    i+=1
    maths.out = mosaicdir+"out_{}_mos.map".format(str(i).zfill(2))
    maths.exp = operate
    maths.options='unmask'
    maths.inp()
    maths.go()

os.rename(mosaicdir+'out_{}_mos.map'.format(str(i).zfill(2)),mosaicdir+'variance_mos.map')
   

In [ ]:
# Calculate transpose of beam matrix multiplied by noise_cov multiplied by image from each beam for each position
# in the final image

maths = lib.miriad('maths')
i=0
for bm in beams:
    operate="'<"+mosaicdir+"image_{}_mos.map>*<".format(str(bm).zfill(2))+mosaicdir+"btci_{}.map>'".format(str(bm).zfill(2))
    if bm!=beams[0]:
        operate=operate[:-1]+"+<"+mosaicdir+"mos_{}.map>'".format(str(i).zfill(2))
    i+=1
    maths.out = mosaicdir+"mos_{}.map".format(str(i).zfill(2))
    maths.exp = operate
    maths.options='unmask,grow'
    maths.inp()
    maths.go()
os.rename(mosaicdir+'mos_{}.map'.format(str(i).zfill(2)),mosaicdir+'mosaic_im.map')


In [ ]:
# Find maximum value of variance map
imstat = lib.miriad('imstat')
imstat.in_="'"+mosaicdir+"variance_mos.map'"
imstat.region="'quarter(1)'"
imstat.axes="'x,y'"
a=imstat.go()

# Always outputs max value at same point
var_max=a[10].split(" ")[-3]


In [ ]:
# Divide image by variance map

maths = lib.miriad('maths')
maths.out = mosaicdir+'mosaic_final.map'
maths.exp="'<"+mosaicdir+"mosaic_im.map>/<"+mosaicdir+"variance_mos.map>'"
maths.mask="'<"+mosaicdir+"variance_mos.map>.gt.0.01*"+str(var_max)+"'"
maths.options='unmask,grow'
maths.inp()
maths.go()


In [ ]:
# Produce mosaic noise map
maths = lib.miriad('maths')
maths.out = mosaicdir+'mosaic_noise.map'
maths.exp="'1./sqrt(<"+mosaicdir+"variance_mos.map>)'"
maths.mask="'<"+mosaicdir+"variance_mos.map>.gt.0.01*"+str(var_max)+"'"
maths.inp()
maths.go()

puthd = lib.miriad('puthd')
puthd.in_=mosaicdir+'mosaic_noise.map/bunit'
puthd.value='JY/BEAM'
puthd.go()

In [ ]:
# Clean up files
for fl in glob.glob(mosaicdir+'*_convol.map'):
    shutil.rmtree(fl)

shutil.rmtree(mosaicdir+'mosaic_im.map')

for fl in glob.glob(mosaicdir+'mos_*.map'):
    shutil.rmtree(fl)

for fl in glob.glob(mosaicdir+'btci_*.map'):
    shutil.rmtree(fl)

In [ ]:
# Write out FITS files
fits = lib.miriad('fits')
fits.op='xyout'
fits.in_=mosaicdir+'mosaic_final.map'
fits.out=mosaicdir+projectid+'_mosaic.fits'
fits.inp()
fits.go()

fits.in_=mosaicdir+'mosaic_noise.map'
fits.out=mosaicdir+projectid+'_mosaic_noise.fits'
fits.inp()
fits.go()